In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path='/content/drive/MyDrive/CSE 234/data/dedup-data-6187_37728.csv'

In [8]:
import pandas as pd


In [16]:
df=pd.read_csv(path)

In [17]:
df.head(10)

,Unnamed: 0,w1,w2,count1,count2,total1,total2,isDuplicate
0,0,n,N,877.0,877.0,158026.0,158026.0,1.0
1,1,n,Y,877.0,877.0,158026.0,158026.0,0.0
2,2,N,n,6295.0,6295.0,158026.0,158026.0,1.0
3,3,y,N,2260.0,2260.0,158026.0,158026.0,0.0
4,5,Y,N,16358.0,16358.0,158026.0,158026.0,0.0
5,6,Y,y,16358.0,16358.0,158026.0,158026.0,1.0
6,9,y,Y,2260.0,2260.0,158026.0,158026.0,1.0
7,10,West Midlands,West Midlands (Region),1934.0,1934.0,50000.0,50000.0,1.0
8,11,Dyfed,Buckinghamshire,39.0,39.0,50000.0,50000.0,0.0
9,12,West Midlands (Region),West Midlands,441.0,441.0,50000.0,50000.0,1.0


In [23]:
df['w1']=df['w1'].apply(lambda x:x.lower())
df['w2']=df['w2'].apply(lambda x:x.lower())
df=df.drop('Unnamed: 0',axis=1)

In [24]:
df.head(10)

,w1,w2,count1,count2,total1,total2,isDuplicate
0,n,n,877.0,877.0,158026.0,158026.0,1.0
1,n,y,877.0,877.0,158026.0,158026.0,0.0
2,n,n,6295.0,6295.0,158026.0,158026.0,1.0
3,y,n,2260.0,2260.0,158026.0,158026.0,0.0
4,y,n,16358.0,16358.0,158026.0,158026.0,0.0
5,y,y,16358.0,16358.0,158026.0,158026.0,1.0
6,y,y,2260.0,2260.0,158026.0,158026.0,1.0
7,west midlands,west midlands (region),1934.0,1934.0,50000.0,50000.0,1.0
8,dyfed,buckinghamshire,39.0,39.0,50000.0,50000.0,0.0
9,west midlands (region),west midlands,441.0,441.0,50000.0,50000.0,1.0


In [25]:
!pip install strsimpy

     |████████████████████████████████| 45 kB 1.5 MB/s 


In [27]:
from strsimpy import Jaccard,Cosine,WeightedLevenshtein

In [31]:
j=Jaccard(2)

In [32]:
j.distance('Hello','Hell')

0.25

In [39]:
j=Jaccard(1)
df['jaccard_1'] = df.apply(lambda x: j.distance(x.w1, x.w2), axis=1)
j=Jaccard(2)
df['jaccard_2'] = df.apply(lambda x: j.distance(x.w1, x.w2), axis=1)
j=Jaccard(3)
df['jaccard_3'] = df.apply(lambda x: j.distance(x.w1, x.w2), axis=1)

In [40]:
df.head(10)

,w1,w2,count1,count2,total1,total2,isDuplicate,jaccard_1,jaccard_2,jaccard_3
0,n,n,877.0,877.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00
1,n,y,877.0,877.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00
2,n,n,6295.0,6295.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00
3,y,n,2260.0,2260.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00
4,y,n,16358.0,16358.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00
5,y,y,16358.0,16358.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00
6,y,y,2260.0,2260.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00
7,west midlands,west midlands (region),1934.0,1934.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45
8,dyfed,buckinghamshire,39.0,39.0,50000.0,50000.0,0.0,0.9375,1.000000,1.00
9,west midlands (region),west midlands,441.0,441.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45


In [43]:
cosine=Cosine(1)
df['cosine_1'] = df.apply(lambda x: cosine.distance(x.w1, x.w2), axis=1)
cosine=Cosine(2)
df['cosine_2'] = df.apply(lambda x: cosine.distance(x.w1, x.w2), axis=1)
cosine=Cosine(3)
df['cosine_3'] = df.apply(lambda x: cosine.distance(x.w1, x.w2), axis=1)

In [44]:
df.head(10)

,w1,w2,count1,count2,total1,total2,isDuplicate,jaccard_1,jaccard_2,jaccard_3,cosine_1,cosine_2,cosine_3
0,n,n,877.0,877.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000
1,n,y,877.0,877.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000
2,n,n,6295.0,6295.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000
3,y,n,2260.0,2260.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000
4,y,n,16358.0,16358.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000
5,y,y,16358.0,16358.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000
6,y,y,2260.0,2260.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000
7,west midlands,west midlands (region),1934.0,1934.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45,0.126515,0.244071,0.25838
8,dyfed,buckinghamshire,39.0,39.0,50000.0,50000.0,0.0,0.9375,1.000000,1.00,0.913289,1.000000,1.00000
9,west midlands (region),west midlands,441.0,441.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45,0.126515,0.244071,0.25838


In [52]:
def get_substitution_cost():
    return 2
def get_insertion_cost():
    return 1
def get_deletion_cost():
    return 1

In [55]:
lev=WeightedLevenshtein()
df['w_levenshtein'] = df.apply(lambda x: lev.distance(x.w1, x.w2), axis=1)

In [57]:
df.head(10)

,w1,w2,count1,count2,total1,total2,isDuplicate,jaccard_1,jaccard_2,jaccard_3,cosine_1,cosine_2,cosine_3,w_levenshtein
0,n,n,877.0,877.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000,0.0
1,n,y,877.0,877.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000,1.0
2,n,n,6295.0,6295.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000,0.0
3,y,n,2260.0,2260.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000,1.0
4,y,n,16358.0,16358.0,158026.0,158026.0,0.0,1.0000,1.000000,1.00,1.000000,1.000000,1.00000,1.0
5,y,y,16358.0,16358.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000,0.0
6,y,y,2260.0,2260.0,158026.0,158026.0,1.0,0.0000,0.000000,0.00,0.000000,0.000000,0.00000,0.0
7,west midlands,west midlands (region),1934.0,1934.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45,0.126515,0.244071,0.25838,9.0
8,dyfed,buckinghamshire,39.0,39.0,50000.0,50000.0,0.0,0.9375,1.000000,1.00,0.913289,1.000000,1.00000,15.0
9,west midlands (region),west midlands,441.0,441.0,50000.0,50000.0,1.0,0.3125,0.428571,0.45,0.126515,0.244071,0.25838,9.0


In [58]:
df1=df[df['isDuplicate']==0].sample(n=7000)

In [59]:
df2=df[df['isDuplicate']==1]

In [60]:
df=pd.concat([df1,df2])

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pyplot as plt


In [98]:
import scikitplot as skplt
import matplotlib.pyplot as plt



##COMBINED

In [79]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['isDuplicate','w1','w2','total1','total2','count1','count2'],axis=1), 
                                                    df['isDuplicate'], test_size=0.3)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94      2147
         1.0       0.95      0.90      0.93      1810

    accuracy                           0.93      3957
   macro avg       0.94      0.93      0.93      3957
weighted avg       0.93      0.93      0.93      3957



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [80]:
from sklearn.svm import SVC
logmodel = SVC()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.90      0.97      0.94      2147
         1.0       0.96      0.88      0.92      1810

    accuracy                           0.93      3957
   macro avg       0.93      0.92      0.93      3957
weighted avg       0.93      0.93      0.93      3957



##JACCARD SIMILARITY


In [75]:
X_train, X_test, y_train, y_test = train_test_split(df[['jaccard_1','jaccard_2','jaccard_3']],df['isDuplicate'], test_size=0.3)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      2107
         1.0       0.95      0.89      0.92      1850

    accuracy                           0.93      3957
   macro avg       0.93      0.92      0.93      3957
weighted avg       0.93      0.93      0.93      3957



In [76]:
from sklearn.svm import SVC
logmodel = SVC()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.94      2107
         1.0       0.95      0.89      0.92      1850

    accuracy                           0.93      3957
   macro avg       0.93      0.93      0.93      3957
weighted avg       0.93      0.93      0.93      3957



In [83]:
X_train, X_test, y_train, y_test = train_test_split(df[['jaccard_1','jaccard_2','jaccard_3','w_levenshtein']],df['isDuplicate'], test_size=0.3)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.94      2103
         1.0       0.95      0.90      0.92      1854

    accuracy                           0.93      3957
   macro avg       0.93      0.93      0.93      3957
weighted avg       0.93      0.93      0.93      3957



In [85]:
from sklearn.svm import SVC
logmodel = SVC()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93      2103
         1.0       0.97      0.85      0.90      1854

    accuracy                           0.92      3957
   macro avg       0.92      0.91      0.91      3957
weighted avg       0.92      0.92      0.92      3957



##COSINE SIMILARITY


In [77]:
X_train, X_test, y_train, y_test = train_test_split(df[['cosine_1','cosine_2','cosine_3']],df['isDuplicate'], test_size=0.3)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93      2090
         1.0       0.94      0.91      0.92      1867

    accuracy                           0.93      3957
   macro avg       0.93      0.93      0.93      3957
weighted avg       0.93      0.93      0.93      3957



In [78]:
from sklearn.svm import SVC
logmodel = SVC()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.92      0.95      0.94      2090
         1.0       0.94      0.91      0.93      1867

    accuracy                           0.93      3957
   macro avg       0.93      0.93      0.93      3957
weighted avg       0.93      0.93      0.93      3957



##LEV SIMILARITY


In [81]:
X_train, X_test, y_train, y_test = train_test_split(df[['w_levenshtein']],df['isDuplicate'], test_size=0.3)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.65      0.64      0.65      2108
         1.0       0.60      0.61      0.61      1849

    accuracy                           0.63      3957
   macro avg       0.63      0.63      0.63      3957
weighted avg       0.63      0.63      0.63      3957



In [82]:
from sklearn.svm import SVC
logmodel = SVC()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.62      0.87      0.73      2108
         1.0       0.73      0.41      0.52      1849

    accuracy                           0.65      3957
   macro avg       0.68      0.64      0.62      3957
weighted avg       0.67      0.65      0.63      3957

